# Install requirements

In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install spacy-conll

     |████████████████████████████████| 12.8MB 254kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 460kB 33.0MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
     |████████████████████████████████| 9.1MB 41.6MB/s 
     |████████████████████████████████| 122kB 42.1MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=2075dd96f40957efe58c3851dff42a8573bb38f048324fee10b7cbc97837b2a1
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 5.0.0
    Uninstalling smart-open-5.0.0:
      Successfully uninstalled smart-open-5.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-

# Download dataset and conll script

In [2]:
!wget -O /content/conll2003.zip https://github.com/esrel/NLU.Lab.2021/blob/master/src/conll2003.zip?raw=true
!wget https://raw.githubusercontent.com/esrel/NLU.Lab.2021/master/src/conll.py

--2021-04-29 17:33:18--  https://github.com/esrel/NLU.Lab.2021/blob/master/src/conll2003.zip?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/esrel/NLU.Lab.2021/raw/master/src/conll2003.zip [following]
--2021-04-29 17:33:18--  https://github.com/esrel/NLU.Lab.2021/raw/master/src/conll2003.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/esrel/NLU.Lab.2021/master/src/conll2003.zip [following]
--2021-04-29 17:33:18--  https://raw.githubusercontent.com/esrel/NLU.Lab.2021/master/src/conll2003.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awa

Unzip the dataset

In [3]:
!unzip conll2003.zip -d /content/dataset/

Archive:  conll2003.zip
  inflating: /content/dataset/dev.txt  
  inflating: /content/dataset/__MACOSX/._dev.txt  
  inflating: /content/dataset/test.txt  
  inflating: /content/dataset/__MACOSX/._test.txt  
  inflating: /content/dataset/train.txt  
  inflating: /content/dataset/__MACOSX/._train.txt  


# Imports

In [4]:
import spacy, nltk
nlp = spacy.load('en_core_web_sm')
import pandas as pd
import conll
from sklearn.metrics import classification_report

# 1) Evaluate spaCy NER on CoNLL 2003 data (provided)

import_dataset(path):  
  * Input: the path of the dataset to load
  * Output: two lists:
    1. text_dataset: contains the lists of sentences of the dataset as text
    2. dataset: contais the pair (token, name entity) for each token, divided in sentences (one list for each sentence)
  * Implementation: it reads the dataset using conll function, for each sentence it extracts the tokens as text or the tuple (token, name entity) 

In [5]:
def import_dataset(path):
  data = conll.read_corpus_conll(path)
  text_dataset = []
  dataset = []
  for t in data:
    sentence = []
    txt = ""
    for t2 in t:
      sentence.append((t2[0].split()[0], t2[0].split()[3]))
      txt += str(t2[0].split()[0]) + " "
    dataset.append(sentence)
    text_dataset.append([txt])
  return text_dataset, dataset

convert_type(ent_type):
  * Input: named entity from spaCy
  * Output: the name entity converted in the dataset format
  * Implementation: assign a specific named entity from the dataset format to each named entity from spaCy

In [65]:
def convert_type(ent_type):
  if(ent_type in ["ORG", "WORK_OF_ART"]):
    return "ORG" 
  if(ent_type in ["GPE", "LOC"]):
    return "LOC"
  if(ent_type in ["LANGUAGE", "NORP", "EVENT", "LAW", "PRODUCT", "MONEY"]):
    return "MISC"
  if(ent_type in ["PERSON", "FAC"]):
    return "PER"
  return ""

convert_spacy(token, parent=None, parent_iob=None):
  * Input: 
    * Token: the token to convert
    * Parent: the parent of the token to use to group the named entity
    * Parent_iob: this tag is used to know if there is a parent in the tree which already has the B tag, if yes the I tag will be assigned to the token, B otherwise.
  * Output: the tags converted in form ```iob-type``` as in the dataset
  * Implementation: 
    * if parent is None it returns just the concatenation between the ```IOB``` tag and the named entity tag
    * if parent is set it returns the named entity from the parent if possible



In [79]:
def convert_spacy(token, parent=None, parent_iob=None): # parent_iob maybe different from the current parent.ent_iob_
  if(parent == None): # exercise 1 usage
    if(convert_type(token.ent_type_) == ""):
      return "O"
    else:
      return f"{token.ent_iob_}-{convert_type(token.ent_type_)}"
  else: # exercise 3 usage
    iob = "I"
    if(parent_iob == "I"):
      iob = "B"
    if(parent.ent_type_ != ""):
      if(convert_type(parent.ent_type_) == ""):
        return "O"
      return f"{iob}-{convert_type(parent.ent_type_)}"
    else:
      if(token.ent_iob_ == "O"):
        return "O"
      else:
        if(convert_type(token.ent_type_) == ""):
          return "O"
        return f"{token.ent_iob_}-{convert_type(token.ent_type_)}"

reconstruct_output(doc, comp=False, ancestors = True):
  * Input: 
    * Doc object from spaCy 
    * comp (compound) flag to set on the third exercise
    * ancestors: this parameter is used in the final experiment of the third point, where I try just the direct parent of the token and not the entire tree
  * Output: list of sentences, each sentence contains the token "reconstructed" as in the dataset
  * Implementation: 
    * given a token it uses whitespace to check if the token is part of a word in the dataset, if yes it concatenates the tokens with the same tag, otherwise the single token is used.  
    * if comp is set to True, the tokens with compound dependency will have the same tag as the first parent with a dependency different from "compound" (if possible) 

In [32]:
def reconstruct_output(doc, comp=False, ancestors = True):
  output = []
  current_token = ""
  current_tag = ""
  first = True
  for token in doc:
    if(first):
        current_tag = convert_spacy(token)
        if((comp) and (token.dep_ == "compound")):
          parent = token.head
          parent_iob = token.head.ent_iob_
          while((parent.dep_ == "compound") and (ancestors)):
            if(parent_iob != "B"):
              parent_iob = parent.head.ent_iob_
            parent = parent.head
          current_tag = convert_spacy(token, parent, parent_iob)
        first = False
    if(not token.whitespace_):
      current_token += token.text
    else:
      current_token += token.text
      output.append((current_token, current_tag))
      first = True
      current_token = ""
      current_tag = ""
  if(not first):
    output.append((current_token, current_tag))
  
  return output

process_dataset(dataset_text, expand, ancestors):
  * Input: the dataset as lists of sentences, expand is a flag used in the third exercise to expand the named entities as well as the ancestors
  * Output: the predicted named entities
  * Implementation: it processes each sentence using nlp and it calls reconstruct_output to format it as in the dataset 

In [33]:
def process_dataset(dataset_text, expand, ancestors):
  pred = []
  for sentence in dataset_text:
    spacy_output = nlp(sentence[0])
    pred.append(reconstruct_output(spacy_output, expand, ancestors))
  return pred

get_accuracy(dataset_text, dataset_refs, expand = False, ancestors = True):
  * Input: 
    * dataset_text: the dataset as lists of sentences (text)
    * dataset_refs: the true named entities from the dataset
    * expand: whether to use the expanded version (ex3) or not
    * ancestors: usually True, just used in the final experiment (ex3)
  * Output:
    * the scikit classification report of spaCy NER on the specified dataset (using the setting on convert_type function)
    * the predictions
  * Implementation: process the dataset and compute the report


In [34]:
def get_accuracy(dataset_text, dataset_refs, expand = False, ancestors = True):
  pred = process_dataset(dataset_text, expand, ancestors)
  predicted = []

  for sentence in pred:
    for token in sentence:
      predicted.append(token[1])
  
  true_labels = []
  for sentence in dataset_refs:
    for token in sentence:
      true_labels.append(token[1])

  report = classification_report(true_labels, predicted)

  return report, pred

# -) Execution

In [11]:
dev_path = '/content/dataset/dev.txt'
train_path = '/content/dataset/train.txt'
test_path = '/content/dataset/test.txt'

Extract the datasets as:
 * *_txt: list of sentences as text
 * *_refs: the true named entities from each dataset

In [12]:
dev_txt, dev_refs = import_dataset(dev_path)
train_txt, train_refs = import_dataset(train_path)
test_txt, test_refs = import_dataset(test_path)

1.1) Compute the token level accuracy for the test set

In [80]:
report_test, pred = get_accuracy(test_txt, test_refs)
print(report_test)

              precision    recall  f1-score   support

       B-LOC       0.77      0.68      0.72      1668
      B-MISC       0.58      0.55      0.57       702
       B-ORG       0.51      0.31      0.38      1661
       B-PER       0.79      0.63      0.70      1617
       I-LOC       0.57      0.53      0.55       257
      I-MISC       0.26      0.36      0.30       216
       I-ORG       0.41      0.52      0.46       835
       I-PER       0.82      0.79      0.80      1156
           O       0.95      0.97      0.96     38554

    accuracy                           0.90     46666
   macro avg       0.63      0.59      0.61     46666
weighted avg       0.90      0.90      0.90     46666



1.2) Compute the chunk level accuracy for the test set using the evaluate function provided by conll.py

In [81]:
results = conll.evaluate(test_refs, pred)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.760,0.610,0.677,1617
ORG,0.460,0.277,0.346,1661
LOC,0.755,0.667,0.708,1668
MISC,0.576,0.546,0.560,702
total,0.663,0.521,0.583,5648


# -) Experiment
Here I was curious about using already tokenized text from the dataset (overriding spaCy tokenizer).  
Despite spaCy's documentation reports that the performance should decrease (due to the fact that the tokenization methods may be different) the perfomance remains similar.

In [69]:
from spacy.tokens import Doc

# function to replace spaCy tokenizer
def get_tokens(sentence):
  return Doc(nlp.vocab, sentence)

In [82]:
nlp.tokenizer = get_tokens

data = conll.read_corpus_conll(test_path)
pred = []

for s in data:
  sentence = []
  for token in s:
    sentence.append(token[0].split()[0])
  doc = nlp(sentence)
  pred.append(reconstruct_output(doc))

predicted = []
for sentence in pred:
  for token in sentence:
    predicted.append(token[1])

true_labels = []
for sentence in test_refs:
  for token in sentence:
    true_labels.append(token[1])

report = classification_report(true_labels, predicted)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.78      0.70      0.74      1668
      B-MISC       0.58      0.55      0.56       702
       B-ORG       0.50      0.30      0.38      1661
       B-PER       0.77      0.61      0.68      1617
       I-LOC       0.60      0.62      0.61       257
      I-MISC       0.27      0.37      0.31       216
       I-ORG       0.41      0.52      0.46       835
       I-PER       0.80      0.76      0.78      1156
           O       0.95      0.97      0.96     38554

    accuracy                           0.90     46666
   macro avg       0.63      0.60      0.61     46666
weighted avg       0.90      0.90      0.90     46666



In [83]:
results = conll.evaluate(test_refs, pred)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.748,0.592,0.661,1617
ORG,0.444,0.273,0.338,1661
LOC,0.766,0.695,0.729,1668
MISC,0.571,0.541,0.556,702
total,0.659,0.522,0.583,5648


# 2) Grouping of Entities.  
Write a function to group recognized named entities using noun_chunks method of spaCy. Analyze the groups in terms of most frequent combinations (i.e. NER types that go together).

group_eintities(sentence):
  * Input: the sentence to process
  * Output: named entities grouped based on noun_chunk
  * Implementation:
    * first a set containing all the sentence entities is created
    * for each noun_chunk its entities are checked if they belong to the main entity set, if yes they will be part of the chunk group
    * the entities added are removed from the main set
    * in the end if the set is not empty, each remaining entity is added to a different new chunk (entities that were not in any chunk)

In [72]:
# I checked whether all the entities of the sentence (doc.ents) are inside chunk.ents.
# there might be new entities inside chunk.ents, they will be discarded, so just the main entities from the sentence will be considered.

def group_entities(sentence):
  doc = nlp(sentence)
  groups = []
  entities = set()

  for ent in doc.ents:
    entities.add(ent)

  for chunk in doc.noun_chunks:
    group = []
    for span in chunk.ents:
      if span in entities:
        group.append(span.root.ent_type_)
        entities.remove(span)
    if(len(group) != 0):
      groups.append(group)

  for ent in entities:
    groups.append([ent.root.ent_type_])

  return groups

get_frequencies(dataset):
  * Input: the dataset where counting the combinations of entities
  * Output: a dict containing the frequencies for each combination
  * Implementation:
    * process each sentence of the dataset and groups its entities using group_entities
    * for each group create a tuple and increase the count of that group (combination) on the dict

In [73]:
def get_frequencies(dataset):
  freq = dict()
  for sentence in dataset:
    groups = group_entities(sentence[0])
    for group in groups:
      group = tuple(group)
      if(group in freq):
        freq[group] += 1
      else:
        freq[group] = 1
  return freq

**Get the frequencies of the test set**  
Print the dictionary containing the frequencies

In [74]:
nlp = spacy.load('en_core_web_sm') # reset the tokenizer if 1.experiment has been run
freq = get_frequencies(test_txt)
freq_ordered = dict(sorted(freq.items(), key=lambda item: item[1], reverse=True))
print("Frequencies:")
for group in freq_ordered:
  print(f"{group} -> {freq_ordered[group]}")

Frequencies:
('CARDINAL',) -> 1624
('GPE',) -> 1255
('PERSON',) -> 1074
('DATE',) -> 997
('ORG',) -> 873
('NORP',) -> 293
('MONEY',) -> 147
('ORDINAL',) -> 111
('TIME',) -> 83
('PERCENT',) -> 81
('EVENT',) -> 58
('LOC',) -> 54
('CARDINAL', 'PERSON') -> 51
('QUANTITY',) -> 51
('NORP', 'PERSON') -> 43
('GPE', 'PERSON') -> 34
('GPE', 'GPE') -> 26
('FAC',) -> 22
('PRODUCT',) -> 22
('ORG', 'PERSON') -> 21
('CARDINAL', 'ORG') -> 19
('CARDINAL', 'NORP') -> 15
('CARDINAL', 'GPE') -> 13
('GPE', 'ORG') -> 13
('LAW',) -> 11
('WORK_OF_ART',) -> 10
('GPE', 'PRODUCT') -> 9
('DATE', 'EVENT') -> 8
('DATE', 'ORG') -> 8
('ORG', 'ORG') -> 8
('PERSON', 'PERSON') -> 8
('NORP', 'ORG') -> 8
('DATE', 'TIME') -> 7
('ORG', 'DATE') -> 6
('LANGUAGE',) -> 6
('GPE', 'DATE') -> 5
('CARDINAL', 'CARDINAL') -> 5
('NORP', 'ORDINAL') -> 5
('ORG', 'GPE') -> 5
('DATE', 'NORP') -> 5
('GPE', 'ORDINAL') -> 4
('ORDINAL', 'PERSON') -> 4
('GPE', 'CARDINAL') -> 4
('ORG', 'NORP') -> 4
('PERSON', 'GPE') -> 4
('CARDINAL', 'DATE') ->

# 3) One of the possible post-processing steps is to fix segmentation errors.  
Write a function that extends the entity span to cover the full noun-compounds. Make use of compound dependency relation.

For this point I reused the get_accuracy function of the first point.  
In this case the expand flag is set to True, this means that to the tokens with compound dependence will be assigned the tag of their parents (if possible).

In [84]:
nlp = spacy.load('en_core_web_sm') # reset the tokenizer if 1.experiment has been run
report_test, pred = get_accuracy(test_txt, test_refs, expand=True)
print(report_test)

              precision    recall  f1-score   support

       B-LOC       0.77      0.67      0.72      1668
      B-MISC       0.57      0.57      0.57       702
       B-ORG       0.43      0.34      0.38      1661
       B-PER       0.62      0.64      0.63      1617
       I-LOC       0.46      0.49      0.48       257
      I-MISC       0.29      0.35      0.32       216
       I-ORG       0.42      0.39      0.40       835
       I-PER       0.81      0.74      0.77      1156
           O       0.95      0.97      0.96     38554

    accuracy                           0.89     46666
   macro avg       0.59      0.57      0.58     46666
weighted avg       0.89      0.89      0.89     46666



In [85]:
results = conll.evaluate(test_refs, pred)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.549,0.581,0.565,1617
ORG,0.332,0.265,0.295,1661
LOC,0.733,0.659,0.694,1668
MISC,0.554,0.560,0.557,702
total,0.548,0.509,0.527,5648


As we can see, using this method, the performance slightly decreases.
***

As final experiment, for curiosity, I try to use just the direct parent of the node and not the whole ancestors.

In [86]:
nlp = spacy.load('en_core_web_sm') # reset the tokenizer if 1.experiment has been run
report_test, pred = get_accuracy(test_txt, test_refs, expand=True, ancestors = False)
print(report_test)

              precision    recall  f1-score   support

       B-LOC       0.77      0.68      0.72      1668
      B-MISC       0.57      0.58      0.57       702
       B-ORG       0.43      0.34      0.38      1661
       B-PER       0.66      0.64      0.65      1617
       I-LOC       0.48      0.50      0.49       257
      I-MISC       0.29      0.34      0.31       216
       I-ORG       0.44      0.39      0.42       835
       I-PER       0.81      0.74      0.78      1156
           O       0.95      0.97      0.96     38554

    accuracy                           0.90     46666
   macro avg       0.60      0.58      0.59     46666
weighted avg       0.89      0.90      0.89     46666



In [87]:
results = conll.evaluate(test_refs, pred)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.587,0.583,0.585,1617
ORG,0.330,0.272,0.298,1661
LOC,0.732,0.668,0.699,1668
MISC,0.552,0.570,0.561,702
total,0.557,0.515,0.535,5648


As we can see the perfomance are similar with a slightly increase at chunk level, however this method does not have much sense since the IOB tag and named entity tag will be chosen without considering the whole span.